In [1]:
import os
from azureml.core import Workspace, Experiment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.data.data_reference import DataReference
from azureml.core.runconfig import RunConfiguration
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
import json

In [2]:
with open('config/config.json', 'r') as f:
    config = json.loads(f.read())
    
subscription_id = config["SUBSCRIPTION_ID"]
resource_group = config["RESOURCE_GROUP"]
workspace_name = config["WORKSPACE_NAME"]
gpu_cluster_name = config["GPU_CLUSTER_NAME"]

ws = Workspace(workspace_name=workspace_name, subscription_id=subscription_id, resource_group=resource_group)

In [3]:
scripts_folder = "scripts"



if gpu_cluster_name in ws.compute_targets:
    gpu_cluster = ws.compute_targets[gpu_cluster_name]
    
    if gpu_cluster and type(gpu_cluster) is AmlCompute:
        print('Compute target found. Using: ' + gpu_cluster_name)
else:
    print("Creating new cluster")
    
    # vm_size parameter below could be modified to one of the RAPIDS-supported VM types
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "Standard_NC6s_v2", min_nodes=1, max_nodes = 1)

    # create the cluster
    gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, provisioning_config)
    gpu_cluster.wait_for_completion(show_output=True)

Compute target found. Using: gpu-todrabas


In [5]:
file_root = 'unswiot'
ds = ws.get_default_datastore()

# data already uploaded to the datastore
data_ref = DataReference(data_reference_name='data', datastore=ds, path_on_datastore=file_root)

In [18]:
run_config = RunConfiguration()
run_config.framework = 'python'
run_config.environment.python.user_managed_dependencies = True
run_config.environment.python.interpreter_path = '/conda/envs/rapids/bin/python'
run_config.target = gpu_cluster_name
run_config.environment.docker.enabled = True
run_config.environment.docker.gpu_support = True
run_config.environment.docker.base_image = "todrabas/mlads_rapids:latest"
# run_config.environment.docker.base_image = "rapidsai/rapidsai:cuda9.2-runtime-ubuntu18.04"
run_config.environment.spark.precache_packages = False
run_config.data_references={'data':data_ref.to_config()}

In [28]:
src = ScriptRunConfig(source_directory=scripts_folder, 
                          script='3_Rapids_flow_classification.py', 
                          arguments = ['--data_dir', str(data_ref)],
                          run_config=run_config
                         )

exp = Experiment(ws, 'rapidstest_iot_flow_gpu')
run = exp.submit(config=src)
run.wait_for_completion(show_output=True)

RunId: rapidstest_iot_flow_gpu_1559603271_3b7eae0f
Web View: https://mlworkspace.azure.ai/portal/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/MLADS_todrabas/providers/Microsoft.MachineLearningServices/workspaces/todrabas_MLADS_WE/experiments/rapidstest_iot_flow_gpu/runs/rapidstest_iot_flow_gpu_1559603271_3b7eae0f

Streaming azureml-logs/80_driver_log.txt

Running NETWORK FLOW on GPU...
###[ Ethernet ]### 
  dst       = 14:cc:20:51:33:ea
  src       = 30:8c:fb:2f:e4:b2
  type      = 0x800
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 142
     id        = 55213
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = 6
     chksum    = 0x7ab3
     src       = 192.168.1.106
     dst       = 52.87.241.159
     \options   \
###[ TCP ]### 
        sport     = 40767
        dport     = 443
        seq       = 91355963
        ack       = 679917399
        dataofs   = 8
        reserved  = 0
        flags     =

{'runId': 'rapidstest_iot_flow_gpu_1559603271_3b7eae0f',
 'target': 'gpu-todrabas',
 'status': 'Completed',
 'startTimeUtc': '2019-06-03T23:08:03.849543Z',
 'endTimeUtc': '2019-06-03T23:08:32.206895Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': '13e3ee37-40d8-4929-b26e-065344ff7415',
  'azureml.git.repository_uri': 'git@github.com:drabastomek/MLADS_RAPIDS.git',
  'mlflow.source.git.repoURL': 'git@github.com:drabastomek/MLADS_RAPIDS.git',
  'azureml.git.branch': 'devel',
  'mlflow.source.git.branch': 'devel',
  'azureml.git.commit': 'ba3ab5b273cbdf8a5bcd3345a4a043542de4442c',
  'mlflow.source.git.commit': 'ba3ab5b273cbdf8a5bcd3345a4a043542de4442c',
  'azureml.git.dirty': 'True'},
 'runDefinition': {'script': '3_Rapids_flow_classification.py',
  'arguments': ['--data_dir', '$AZUREML_DATAREFERENCE_data'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'gpu-todrabas',
  'dataReferences': {'data': {'dataStore